# Practical Machine Learning for Physicists
## Week 4 Exercise - Part 1


### Task 1:
Design, implement and test a neural network utilising a single convolutional layer (use as many other non convolutional layers as you need) to classify the MNIST handwritten digits. What is the maximum test accuracry you can achieve using a single convolutional layer?

#### Practicalities
You should use this notebook for your work and upload it to both Moodle and CoCalc. You are expected to use TensorFlow and Keras to complete these takss. The notebook should be self-contained and able to be executed if necessary. Marks will be awarded for (roughly equally weighted):
- Overall notebook clarity (both in terms of good coding practice and coherent discussion)
- Network performance (how well does your classifier do?)
- Network efficiency (how does your network compare to the optimum networks for this task?)
- Network training (do you do a good job of traning your network?)


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math

# TensorFlow and tf.keras
import tensorflow as tf
import keras
import keras.layers
import keras.datasets.mnist

import matplotlib.style #Some style nonsense
import matplotlib as mpl #Some more style nonsense


#Set default figure size
#mpl.rcParams['figure.figsize'] = [12.0, 8.0] #Inches... of course it is inches
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi']=200 # dots per inch

#Useful for debugging problems
print(tf.__version__)

2.11.0


Load MNIST data, re-scale and check the shape of the data:

In [3]:
# load data
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# re-scale inputs
train_images=train_images/255.0
test_images=test_images/255.0

# check shape of datasets
print("Shape of training images:",train_images.shape)
print("Length of training set labels:",len(train_labels))
print("First label:",train_labels[0])
print("Shape of testing images:",test_images.shape)
print("Length of testing set labels:",len(test_labels))

image_x = len(train_images[0,:,0])
image_y = len(train_images[0,0,:])

Shape of training images: (60000, 28, 28)
Length of training set labels: 60000
First label: 5
Shape of testing images: (10000, 28, 28)
Length of testing set labels: 10000


Define and compile model:

In [5]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(15))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1556 - accuracy: 0.9542 - val_loss: 0.0630 - val_accuracy: 0.9795
Epoch 2/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0524 - accuracy: 0.9840 - val_loss: 0.0559 - val_accuracy: 0.9813
Epoch 3/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0339 - accuracy: 0.9893 - val_loss: 0.0434 - val_accuracy: 0.9852
Epoch 4/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0217 - accuracy: 0.9931 - val_loss: 0.0496 - val_accuracy: 0.9853
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0151 - accuracy: 0.9951 - val_loss: 0.0454 - val_accuracy: 0.9863
Epoch 6/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0107 - accuracy: 0.9966 - val_loss: 0.0391 - val_accuracy: 0.9889
Epoch 7/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0085 - accuracy: 0.9973 - val_loss: 0.0490 - val_ac

### Task 2:
Design, implement and test a neural network utitlising multiple convolutional layers (again use as many other non convolutinal laters as you need) to classify the MNIST handwritten digits. What is the maximum test accuracry you can achieve using as many convolutional layers as you like?

In [7]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(15))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1399 - accuracy: 0.9571 - val_loss: 0.0467 - val_accuracy: 0.9861
Epoch 2/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0460 - accuracy: 0.9858 - val_loss: 0.0354 - val_accuracy: 0.9891
Epoch 3/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0320 - accuracy: 0.9895 - val_loss: 0.0343 - val_accuracy: 0.9880
Epoch 4/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0230 - accuracy: 0.9927 - val_loss: 0.0291 - val_accuracy: 0.9907
Epoch 5/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0175 - accuracy: 0.9947 - val_loss: 0.0397 - val_accuracy: 0.9877
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0126 - accuracy: 0.9961 - val_loss: 0.0339 - val_accuracy: 0.9900
Epoch 7/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0111 - accuracy: 0.9964 - val_loss: 0.0363 - val_ac